## Introductie 

Deze les behandelt: 
- Wat functieaanroepen zijn en hun gebruiksgevallen 
- Hoe een functieaanroep te maken met OpenAI 
- Hoe een functieaanroep te integreren in een applicatie 

## Leerdoelen 

Na het voltooien van deze les weet je hoe en begrijp je: 

- Het doel van het gebruik van functieaanroepen 
- Het instellen van functieaanroepen met de OpenAI-service 
- Het ontwerpen van effectieve functieaanroepen voor het gebruiksgeval van je applicatie


## Functieaanroepen begrijpen

Voor deze les willen we een functie bouwen voor onze educatieve startup waarmee gebruikers een chatbot kunnen gebruiken om technische cursussen te vinden. We zullen cursussen aanbevelen die passen bij hun vaardigheidsniveau, huidige rol en technologie van interesse.

Om dit te voltooien gebruiken we een combinatie van:
 - `OpenAI` om een chatervaring voor de gebruiker te creëren
 - `Microsoft Learn Catalog API` om gebruikers te helpen cursussen te vinden op basis van het verzoek van de gebruiker
 - `Function Calling` om de query van de gebruiker te nemen en naar een functie te sturen om het API-verzoek te doen.

Om te beginnen, laten we kijken waarom we functieaanroepen in de eerste plaats zouden willen gebruiken:

print("Berichten in volgende aanvraag:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # krijg een nieuw antwoord van GPT waar het de functieantwoord kan zien


print(second_response.choices[0].message)


### Waarom Functieaanroepen

Als je een andere les in deze cursus hebt voltooid, begrijp je waarschijnlijk de kracht van het gebruik van Large Language Models (LLM's). Hopelijk zie je ook enkele van hun beperkingen.

Functieaanroepen is een functie van de OpenAI Service die is ontworpen om de volgende uitdagingen aan te pakken:

Inconsistente Responsopmaak:
- Voor functieaanroepen waren de reacties van een groot taalmodel ongestructureerd en inconsistent. Ontwikkelaars moesten complexe validatiecode schrijven om elke variatie in de output af te handelen.

Beperkte Integratie met Externe Gegevens:
- Voor deze functie was het moeilijk om gegevens uit andere delen van een applicatie in een chatcontext te verwerken.

Door responsformaten te standaardiseren en naadloze integratie met externe gegevens mogelijk te maken, vereenvoudigt functieaanroepen de ontwikkeling en vermindert het de noodzaak voor extra validatielogica.

Gebruikers konden geen antwoorden krijgen zoals "Wat is het huidige weer in Stockholm?". Dit komt omdat modellen beperkt waren tot de tijd waarop de data getraind was.

Laten we naar het onderstaande voorbeeld kijken dat dit probleem illustreert:

Stel dat we een database van studentgegevens willen maken zodat we hen de juiste cursus kunnen voorstellen. Hieronder hebben we twee beschrijvingen van studenten die erg vergelijkbaar zijn in de gegevens die ze bevatten.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

We willen dit naar een LLM sturen om de gegevens te parseren. Dit kan later in onze applicatie worden gebruikt om dit naar een API te sturen of op te slaan in een database.

Laten we twee identieke prompts maken waarin we de LLM instrueren over welke informatie we geïnteresseerd zijn:


We willen dit naar een LLM sturen om de onderdelen te analyseren die belangrijk zijn voor ons product. Zo kunnen we twee identieke prompts maken om de LLM te instrueren:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Na het maken van deze twee prompts sturen we ze naar de LLM met behulp van `openai.ChatCompletion`. We slaan de prompt op in de variabele `messages` en wijzen de rol toe aan `user`. Dit is om een bericht van een gebruiker dat naar een chatbot wordt geschreven na te bootsen.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Nu kunnen we beide verzoeken naar de LLM sturen en de reactie die we ontvangen onderzoeken.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Hoewel de prompts hetzelfde zijn en de beschrijvingen vergelijkbaar, kunnen we verschillende formaten van de eigenschap `Grades` krijgen.

Als je de bovenstaande cel meerdere keren uitvoert, kan het formaat `3.7` of `3.7 GPA` zijn.

Dit komt omdat de LLM ongestructureerde gegevens in de vorm van de geschreven prompt neemt en ook ongestructureerde gegevens teruggeeft. We moeten een gestructureerd formaat hebben zodat we weten wat we kunnen verwachten bij het opslaan of gebruiken van deze gegevens.

Door functioneel aan te roepen, kunnen we ervoor zorgen dat we gestructureerde gegevens terugkrijgen. Bij het gebruik van functioneel aanroepen roept de LLM eigenlijk geen functies aan of voert deze uit. In plaats daarvan creëren we een structuur die de LLM moet volgen voor zijn antwoorden. We gebruiken die gestructureerde antwoorden vervolgens om te weten welke functie we in onze toepassingen moeten uitvoeren.


![Function Calling Flow Diagram](../../../../translated_images/nl/Function-Flow.083875364af4f4bb.png)


We kunnen dan nemen wat wordt geretourneerd door de functie en dit terugsturen naar de LLM. De LLM zal vervolgens reageren met natuurlijke taal om de vraag van de gebruiker te beantwoorden.


### Gebruikssituaties voor het gebruik van functieaanroepen

**Externe Hulpmiddelen Aanroepen**  
Chatbots zijn uitstekend in het geven van antwoorden op vragen van gebruikers. Door gebruik te maken van functieaanroepen kunnen de chatbots berichten van gebruikers gebruiken om bepaalde taken uit te voeren. Bijvoorbeeld, een student kan de chatbot vragen om "Stuur een e-mail naar mijn docent met de mededeling dat ik meer hulp nodig heb bij dit vak". Dit kan een functieaanroep doen naar `send_email(to: string, body: string)`

**API- of Databasequery's Maken**  
Gebruikers kunnen informatie vinden met natuurlijke taal die wordt omgezet in een geformatteerde query of API-verzoek. Een voorbeeld hiervan kan een docent zijn die vraagt "Wie zijn de studenten die de laatste opdracht hebben voltooid" wat een functie kan aanroepen genaamd `get_completed(student_name: string, assignment: int, current_status: string)`

**Gestructureerde Gegevens Maken**  
Gebruikers kunnen een tekstblok of CSV nemen en de LLM gebruiken om belangrijke informatie eruit te halen. Bijvoorbeeld, een student kan een Wikipedia-artikel over vredesakkoorden omzetten om AI-flashcards te maken. Dit kan gedaan worden door een functie te gebruiken genaamd `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Je eerste functieaanroep maken

Het proces van het maken van een functieaanroep omvat 3 hoofd stappen:  
1. Het aanroepen van de Chat Completions API met een lijst van je functies en een gebruikersbericht  
2. Het lezen van de reactie van het model om een actie uit te voeren, bijvoorbeeld het uitvoeren van een functie of API-aanroep  
3. Nog een keer de Chat Completions API aanroepen met de reactie van je functie om die informatie te gebruiken om een antwoord aan de gebruiker te maken.


![Stroom van een Functieaanroep](../../../../translated_images/nl/LLM-Flow.3285ed8caf4796d7.png)


### Elementen van een functieaanroep 

#### Gebruikersinvoer 

De eerste stap is het maken van een gebruikersbericht. Dit kan dynamisch worden toegewezen door de waarde van een tekstinvoer te nemen of je kunt hier een waarde toewijzen. Als dit je eerste keer is dat je met de Chat Completions API werkt, moeten we de `role` en de `content` van het bericht definiëren. 

De `role` kan `system` zijn (regels maken), `assistant` (het model) of `user` (de eindgebruiker). Voor functieaanroepen zullen we dit toewijzen als `user` en een voorbeeldvraag. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Functies maken. 

Vervolgens definiëren we een functie en de parameters van die functie. We gebruiken hier slechts één functie genaamd `search_courses`, maar je kunt meerdere functies maken.

**Belangrijk** : Functies worden opgenomen in het systeembericht aan de LLM en worden meegerekend in het aantal beschikbare tokens dat je hebt. 


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definities** 

De functiedefinitie-structuur heeft meerdere niveaus, elk met zijn eigen eigenschappen. Hier is een overzicht van de geneste structuur:

**Topniveau Functie-eigenschappen:**

`name` - De naam van de functie die we willen laten aanroepen.

`description` - Dit is de beschrijving van hoe de functie werkt. Hier is het belangrijk om specifiek en duidelijk te zijn.

`parameters` - Een lijst van waarden en het formaat dat je wilt dat het model produceert in zijn antwoord.

**Eigenschappen van het Parameters-object:**

`type` - Het datatype van het parameters-object (meestal "object").

`properties` - Lijst van de specifieke waarden die het model zal gebruiken voor zijn antwoord.

**Eigenschappen van individuele parameters:**

`name` - Impliciet gedefinieerd door de eigenschapssleutel (bijv. "role", "product", "level").

`type` - Het datatype van deze specifieke parameter (bijv. "string", "number", "boolean").

`description` - Beschrijving van de specifieke parameter.

**Optionele eigenschappen:**

`required` - Een array die aangeeft welke parameters vereist zijn om de functie-aanroep te voltooien.


### De functieaanroep maken  
Na het definiëren van een functie moeten we deze nu opnemen in de aanroep naar de Chat Completion API. Dit doen we door `functions` toe te voegen aan het verzoek. In dit geval `functions=functions`.  

Er is ook een optie om `function_call` in te stellen op `auto`. Dit betekent dat we de LLM laten beslissen welke functie moet worden aangeroepen op basis van het gebruikersbericht in plaats van dit zelf toe te wijzen.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Laten we nu naar de reactie kijken en zien hoe deze is opgemaakt:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Je kunt zien dat de naam van de functie wordt aangeroepen en dat de LLM op basis van het bericht van de gebruiker de gegevens heeft kunnen vinden die passen bij de argumenten van de functie.


## 3.Integreren van Functieaanroepen in een Applicatie. 


Nadat we de geformatteerde reactie van de LLM hebben getest, kunnen we deze nu integreren in een applicatie. 

### De stroom beheren 

Om dit in onze applicatie te integreren, laten we de volgende stappen nemen: 

Eerst maken we de oproep naar de OpenAI-diensten en slaan het bericht op in een variabele genaamd `response_message`. 


In [ ]:
response_message = response.choices[0].message

Nu zullen we de functie definiëren die de Microsoft Learn API aanroept om een lijst met cursussen op te halen:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Als beste praktijk zullen we vervolgens kijken of het model een functie wil aanroepen. Daarna zullen we een van de beschikbare functies aanmaken en deze koppelen aan de functie die wordt aangeroepen.  
Vervolgens nemen we de argumenten van de functie en koppelen deze aan de argumenten van het LLM.

Ten slotte voegen we het functie-aanroepbericht en de waarden die zijn geretourneerd door het `search_courses`-bericht toe. Dit geeft het LLM alle informatie die het nodig heeft om op de gebruiker te reageren met natuurlijke taal.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Nu zullen we het bijgewerkte bericht naar de LLM sturen zodat we een natuurlijke taalreactie kunnen ontvangen in plaats van een API JSON-geformatteerd antwoord.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code-uitdaging

Goed gedaan! Om je kennis van OpenAI Function Calling voort te zetten, kun je bouwen: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Meer parameters van de functie die leerlingen kunnen helpen meer cursussen te vinden. Je kunt de beschikbare API-parameters hier vinden:  
 - Maak een andere functieaanroep die meer informatie van de leerling vraagt, zoals hun moedertaal  
 - Maak foutafhandeling wanneer de functieaanroep en/of API-aanroep geen geschikte cursussen retourneert


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsdienst [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u er rekening mee te houden dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet als de gezaghebbende bron worden beschouwd. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
